# <center>Big Data &ndash; Exercises &ndash; Solution</center>
## <center>Fall 2021 &ndash; Week 9 &ndash; ETH Zurich</center>
## <center>Spark Dataframes and Spark SQL, Moodle exercise</center>

# Preparation for the moodle exercise in Spark

In this jupyter notebook we are going to make the preprocessing part of the dataset that is going to be used in the graded exercise of this week.
It will be the same language game dataset as in exercise08.

1. Change to exercise09 repository

2. Start docker <br>
```docker-compose up -d```

3. Getting the data:
Follow the procedure that is described below. The dataset can be found here: http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2. 

More specifically do the following:
- download the data      :<br> ```wget http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2```
- extract the data       :<br> ```tar -jxvf confusion-2014-03-02.tbz2```

4. copy the data to hdfs :<br> ```docker cp confusion-2014-03-02/confusion-2014-03-02.json jupyter:/home/jovyan/work``` <br>
(Copying the data to hdfs needs to be done only once and it might take 1-2 minutes.)

## More Info about the data
You can find more information about the dataset (as well as the schema and examples) in this link: http://lars.yencken.org/datasets/languagegame/

## Instructions:

In every query we ask you for three quantities: the query itself, the result of the query as well as the productivity time. That means the development time of each query (time elapsed before you start writing the query, and the time at which the correct, final query is ready). Note that the time part of every question is optional and not graded. In order to make easier the time recording we created two functions that do it automatically. Run the cell below in order to import the functions into the current notebook. Then before each query we will have a ```start_exercise()``` cell that you have to run in order to start time recording. After you have finished your query and you are sure about the answer run the ```finish_exercise()``` one to get the time measurement. 

In [1]:
import time

def start_exercise():
    global last
    last = time.time()
    
def finish_exercise():
    global last
    print("This exercise took {0}s".format(int(time.time()-last)))

## <center>1. Spark Dataframes</center>

Write queries for the same questions as last week, but this time using Spark Dataframes operations (the data loading will take a couple minutes)

### 1.0. Data preprocessing

In [2]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

path = "confusion-2014-03-02.json"
dataset = spark.read.json(path).cache()

In [3]:
#test it out
dataset.limit(3).show()

+--------------------+-------+----------+---------+--------------------+---------+
|             choices|country|      date|    guess|              sample|   target|
+--------------------+-------+----------+---------+--------------------+---------+
|[Maori, Mandarin,...|     AU|2013-08-19|Norwegian|48f9c924e0d98c959...|Norwegian|
|[Danish, Dinka, K...|     AU|2013-08-19|    Dinka|af5e8f27cef9e689a...|    Dinka|
|[German, Hungaria...|     AU|2013-08-19|  Turkish|509c36eb58dbce009...|   Samoan|
+--------------------+-------+----------+---------+--------------------+---------+



## Assignment 1
Find the number of games where the guessed language is correct (meaning equal to the target one) and that language is Maori.

In [4]:
start_exercise()

In [5]:
dataset.filter("target = 'Maori' and target = guess").count()

290818

In [6]:
finish_exercise()

This exercise took 7s


## Assignment 2
Return the number of distinct "target" languages.

In [7]:
start_exercise()

In [8]:
dataset.select("target").distinct().count()

78

In [9]:
finish_exercise()

This exercise took 11s


## Assignment 3
Return the sample IDs (i.e., the *sample* field) of the bottom three games where the guessed language is incorrect (not equal to the target one) ordered by date (ascending), then by language (descending), then by country (ascending).

In [10]:
start_exercise()

In [11]:
dataset.select("sample").filter("guess != target").orderBy(
    dataset["date"].asc(), dataset["target"].desc(), dataset["country"].desc()
).limit(3).collect()

[Row(sample='00b85faa8b878a14f8781be334deb137'),
 Row(sample='efcd813daec1c836d9f030b30caa07ce'),
 Row(sample='efcd813daec1c836d9f030b30caa07ce')]

In [12]:
finish_exercise()

This exercise took 13s


## Assignment 4
Aggregate all games by country and target language, counting the number of guesses for each group and return the frequencies of the three most frequent country/language combinations.

In [13]:
start_exercise()

In [14]:
from pyspark.sql.functions import desc

dataset.groupBy(["country", "target"]).count().orderBy(desc("count")).select("count").limit(3).collect()

[Row(count=112934), Row(count=112007), Row(count=110919)]

In [15]:
finish_exercise()

This exercise took 12s


## Assignment 5
Find the fraction (between 0 and 1) of games where (the answer was correct && the correct guess was the second choice amongst the array of possible answers)

Please write the fraction rounding to 4 decimals (eg. 0.3320)

In [16]:
start_exercise()

In [17]:
dataset.filter("choices[1] = target and target = guess").count() / float(dataset.count())

0.25603983084476356

In [18]:
finish_exercise()

This exercise took 10s


## Assignment 6
Sort the languages by increasing overall percentage of correct guesses and return the first three languages.

In [19]:
start_exercise()

In [20]:
from pyspark.sql.functions import asc

correct_c = dataset.filter("target = guess").groupBy("target").count().withColumnRenamed("count", "correct_guesses")
total_c = dataset.groupBy("target").count().withColumnRenamed("count", "total_guesses")
joined_df = correct_c.join(total_c, "target")
joined_df.select("*", 
    (joined_df["correct_guesses"] / joined_df["total_guesses"]).alias("percentage")).orderBy(asc("percentage")).limit(3).show()

+-------+---------------+-------------+------------------+
| target|correct_guesses|total_guesses|        percentage|
+-------+---------------+-------------+------------------+
| French|         325430|       346851|0.9382414927447232|
| German|         315271|       342774|0.9197634593055483|
|Spanish|         304147|       339585|0.8956432115670598|
+-------+---------------+-------------+------------------+



In [21]:
finish_exercise()

This exercise took 14s


## Assignment 7
Return the number of games played on the first day.

In [22]:
start_exercise()

In [23]:
min_date = dataset.agg({"date": "min"}).collect()[0]["min(date)"]
dataset.filter(dataset["date"] == min_date).count()

65653

In [24]:
finish_exercise()

This exercise took 8s


## <center>2. Spark SQL</center>

Write Spark SQL queries for the same questions as earlier.

### 2.0. Data preprocessing

In [ ]:
!pip install sparksql-magic

In [38]:
%load_ext sparksql_magic

In [31]:
path = "confusion-2014-03-02.json"
dataset = spark.read.json(path).cache()
dataset.registerTempTable("dataset")

In [40]:
%%sparksql
-- test it out
SELECT *
FROM dataset
LIMIT 3

choices,country,date,guess,sample,target
"['Maori', 'Mandarin', 'Norwegian', 'Tongan']",AU,2013-08-19,Norwegian,48f9c924e0d98c959d8a6f1862b3ce9a,Norwegian
"['Danish', 'Dinka', 'Khmer', 'Lao']",AU,2013-08-19,Dinka,af5e8f27cef9e689a070b8814dcc02c3,Dinka
"['German', 'Hungarian', 'Samoan', 'Turkish']",AU,2013-08-19,Turkish,509c36eb58dbce009ccf93f375358d53,Samoan


## Assignment 1
Find the number of games where the guessed language is correct (meaning equal to the target one) and that language is Maori.

In [41]:
start_exercise()

In [42]:
%%sparksql
-- write you query here
SELECT count(*) FROM dataset
WHERE target == "Maori" 
AND target == guess

count(1)
290818


In [43]:
finish_exercise()

This exercise took 3s


## Assignment 2
Return the number of distinct "target" languages.

In [44]:
start_exercise()

In [45]:
%%sparksql
-- write you query here
SELECT count(distinct(target))
FROM dataset

count(DISTINCT target)
78


In [46]:
finish_exercise()

This exercise took 6s


## Assignment 3
Return the sample IDs (i.e., the *sample* field) of the bottom three games where the guessed language is incorrect (not equal to the target one) ordered by date (ascending), then by language (descending), then by country (ascending).

In [47]:
start_exercise()

In [48]:
%%sparksql
SELECT sample
FROM dataset as d
WHERE guess != target
ORDER BY d.date asc, target desc, country asc
LIMIT 3

sample
00b85faa8b878a14f8781be334deb137
efcd813daec1c836d9f030b30caa07ce
efcd813daec1c836d9f030b30caa07ce


In [49]:
finish_exercise()

This exercise took 11s


## Assignment 4
Aggregate all games by country and target language, counting the number of guesses for each group and return the frequencies of the three most frequent country/language combinations.

In [50]:
start_exercise()

In [51]:
%%sparksql
SELECT count(guess)
FROM dataset 
GROUP BY country, target
ORDER BY count(guess) desc
LIMIT 3

count(guess)
112934
112007
110919


In [52]:
finish_exercise()

This exercise took 13s


## Assignment 5
Find the fraction (between 0 and 1) of games where (the answer was correct && the correct guess was the second choice amongst the array of possible answers)

Please write the fraction rounding to 4 decimals (eg. 0.3320)

In [53]:
start_exercise()

In [54]:
%%sparksql
SELECT count(*) / 
    (SELECT count(*) as count_all
    FROM dataset)
FROM dataset
WHERE choices[1] = target AND target = guess

(CAST(count(1) AS DOUBLE) / CAST(scalarsubquery() AS DOUBLE))
0.25603983084476356


In [55]:
finish_exercise()

This exercise took 9s


## Assignment 6
Sort the languages by increasing overall percentage of correct guesses and return the first three languages.

In [56]:
start_exercise()

In [57]:
%%sparksql
SELECT target 
FROM (
    SELECT target, count(*) as correct_guesses
    FROM dataset
    WHERE target = guess
    GROUP BY target
) JOIN 
(
    SELECT target, count(*) as total_guesses
    FROM dataset
    GROUP BY target
) USING(target)
ORDER BY correct_guesses/total_guesses asc
LIMIT 3

target
French
German
Spanish


In [58]:
finish_exercise()

This exercise took 12s


## Assignment 7
Return the number of games played on the first day.

In [59]:
start_exercise()

In [60]:
%%sparksql
SELECT count(*) 
FROM dataset
WHERE date = 
(
    SELECT min(date)
    FROM dataset
)

count(1)
65653


In [61]:
finish_exercise()

This exercise took 7s
